In [1]:
import glob
import numpy as np
from os.path import exists
import pandas as pd
from PIL import Image
import pygame
import random
import time
import scipy
import sys

pygame 2.1.2 (SDL 2.0.18, Python 3.8.8)
Hello from the pygame community. https://www.pygame.org/contribute.html


In [2]:
OUTPUT_FILENAME = 'output/participant_test_3.txt'
NO_TRIALS = 1000

In [3]:
df_img_rgb = pd.read_csv('input/munsell_rgb.csv') * 255

In [4]:
assert len(df_img_rgb[df_img_rgb['red'] < 0]) == 0
assert len(df_img_rgb[df_img_rgb['green'] < 0]) == 0
assert len(df_img_rgb[df_img_rgb['blue'] < 0]) == 0

assert len(df_img_rgb[df_img_rgb['red'] > 255]) == 0
assert len(df_img_rgb[df_img_rgb['green'] > 255]) == 0
assert len(df_img_rgb[df_img_rgb['blue'] > 255]) == 0

In [5]:
print(320*319*318/(3*2))

5410240.0


In [6]:
comb_s = pd.read_csv('input/combinations.csv').values.tolist()

In [7]:
def load_trial(trial_no):
    global color1, color2, color3, im, bgcolor
    
    comb = comb_s[trial_no]
    img_rgb_first_color = df_img_rgb.iloc[comb[0]].to_list()
    img_rgb_second_color = df_img_rgb.iloc[comb[1]].to_list()
    img_rgb_third_color = df_img_rgb.iloc[comb[2]].to_list()

    bgcolor = np.clip([256]*3, 0, 255).astype(np.uint8)
    color1 = np.array(img_rgb_first_color).astype(np.uint8)
    color2 = np.array(img_rgb_second_color).astype(np.uint8)
    color3 = np.array(img_rgb_third_color).astype(np.uint8)
    
    im = np.zeros((RES, RES, 3)).astype(np.uint8) + bgcolor
    im[RES//2-SWATCH//2:RES//2+SWATCH//2, 128:128+SWATCH] = color1
    im[RES//4-SWATCH//2:RES//4+SWATCH//2, 512:512+SWATCH] = color2
    im[3*RES//4-SWATCH//2:3*RES//4+SWATCH//2, 512:512+SWATCH] = color3

In [8]:
RES = 1024
SWATCH = 256

file_exists = exists(OUTPUT_FILENAME)
if file_exists is True:
    print('Repeated file')
    sys.exit()

pygame.init()
pygame.font.init()
font = pygame.font.SysFont("Arial", 36)
screen = pygame.display.set_mode((RES, RES), 0, 24)
    
t0 = time.time()
trial_counter = 0
load_trial(trial_counter)
trial_counter += 1

done = False
counter = 0
while not done:
    for event in pygame.event.get():
        if event.type == pygame.KEYUP:
            if event.key == pygame.K_q:
                done = True
        if event.type == pygame.MOUSEBUTTONDOWN:
            mx = event.pos[0]
            my = event.pos[1]
            if event.button == 1:
                t1 = time.time()
                if my>=512 and my<=512+SWATCH:
                    if mx>=RES//4-SWATCH//2 and mx<=RES//4+SWATCH//2:
                        f = open(OUTPUT_FILENAME, "a")
                        f.write("%d,%d,%d,%d,%d,%d,%d,%d,%d,%d,%d,%d,%.6g,%d\n" % 
                                (color1[0], color1[1], color1[2],
                                 color2[0], color2[1], color2[2],
                                 color3[0], color3[1], color3[2],
                                 bgcolor[0], bgcolor[1], bgcolor[2],
                                 t1-t0, 0))
                        f.close()
                        load_trial(trial_counter)
                        trial_counter += 1
                        t0 = t1
                        
                        if counter == NO_TRIALS:
                            pygame.quit()
                        else:
                            counter += 1

                    if mx>=3*RES//4-SWATCH//2 and mx<=3*RES//4+SWATCH//2:
                        f = open(OUTPUT_FILENAME, "a")
                        f.write("%d,%d,%d,%d,%d,%d,%d,%d,%d,%d,%d,%d,%.6g,%d\n" % 
                                (color1[0], color1[1], color1[2],
                                 color2[0], color2[1], color2[2],
                                 color3[0], color3[1], color3[2],
                                 bgcolor[0], bgcolor[1], bgcolor[2],
                                 t1-t0, 1))
                        f.close()
                        load_trial(trial_counter)
                        trial_counter += 1
                        t0 = t1

                        if counter == NO_TRIALS:
                            pygame.quit()
                        else:
                            counter += 1

    s_image = pygame.pixelcopy.make_surface(im)
    fontim = font.render(f'{counter+1} / 200', True, [0, 0, 0])

    if counter + 1 < 10:
        s_image.blit(fontim, (RES-130, 0))
    elif counter + 1 < 100:
        s_image.blit(fontim, (RES-150, 0))
    else:
        s_image.blit(fontim, (RES-170, 0))
        
    screen.blit(s_image, (0, 0))

    pygame.display.update()   
    time.sleep(0.001)
                    
pygame.quit()

In [9]:
%tb

No traceback available to show.
